# Import de la base de données

In [1]:
from pprint import pprint as pp
from pymongo import MongoClient
import pandas as pd

client = MongoClient('localhost', 27017)
db = client["devoir"]
df = pd.read_csv("Video_Games.csv")
db.videogames.drop()
db.videogames.insert_many(df.to_dict(orient="records"))

for collections in db.list_collection_names():
    print(collections, db[collections].count_documents({}), '\n')
    pp(db[collections].find_one())


videogames 16719 

{'Critic_Count': 51.0,
 'Critic_Score': 76.0,
 'Developer': 'Nintendo',
 'EU_Sales': 28.96,
 'Genre': 'Sports',
 'Global_Sales': 82.53,
 'JP_Sales': 3.77,
 'NA_Sales': 41.36,
 'Name': 'Wii Sports',
 'Other_Sales': 8.45,
 'Platform': 'Wii',
 'Publisher': 'Nintendo',
 'Rating': 'E',
 'User_Count': 322.0,
 'User_Score': '8',
 'Year_of_Release': 2006.0,
 '_id': ObjectId('628cf517aace3e4978f0fc33')}


# Les 5 jeux de Racing les plus vendus de l'histoire

In [2]:
l = list(db.videogames.find(
    {"Genre": "Racing"},
    {"_id": 0, "Name": 1, "Publisher" : 1, "Year_of_Release": 1, "Global_Sales": 1})
    .sort([("Global_Sales", -1)])
    .limit(5))
pp(l)

[{'Global_Sales': 35.52,
  'Name': 'Mario Kart Wii',
  'Publisher': 'Nintendo',
  'Year_of_Release': 2008.0},
 {'Global_Sales': 23.21,
  'Name': 'Mario Kart DS',
  'Publisher': 'Nintendo',
  'Year_of_Release': 2005.0},
 {'Global_Sales': 14.98,
  'Name': 'Gran Turismo 3: A-Spec',
  'Publisher': 'Sony Computer Entertainment',
  'Year_of_Release': 2001.0},
 {'Global_Sales': 12.66,
  'Name': 'Mario Kart 7',
  'Publisher': 'Nintendo',
  'Year_of_Release': 2011.0},
 {'Global_Sales': 11.66,
  'Name': 'Gran Turismo 4',
  'Publisher': 'Sony Computer Entertainment',
  'Year_of_Release': 2004.0}]


# Les années avec le plus de ventes de jeux de racings

In [4]:
rep =  db.videogames.aggregate([
    { "$match": { "Genre": "Racing" } },
    {
        "$group": {
            "_id": "$Year_of_Release",
            "TotalSales": {"$sum": "$Global_Sales"}
        }
    },
    { "$project": {
        "_id": 0,
        "Year": "$_id",
        "TotalSales": 1
    }},
    {
        "$sort": { "TotalSales": -1}
    },
    {
        "$limit": 10
    }
])
for r in rep:
    print(r)


{'TotalSales': 70.03, 'Year': 2008.0}
{'TotalSales': 56.03, 'Year': 2005.0}
{'TotalSales': 55.81, 'Year': 2001.0}
{'TotalSales': 52.19, 'Year': 2003.0}
{'TotalSales': 47.83, 'Year': 2004.0}
{'TotalSales': 38.87, 'Year': 2007.0}
{'TotalSales': 37.27, 'Year': 1999.0}
{'TotalSales': 34.83, 'Year': 2011.0}
{'TotalSales': 34.36, 'Year': 2010.0}
{'TotalSales': 33.86, 'Year': 2006.0}


# Les bestsellers Racing de chaque année, par ordre de ventes totales

In [7]:
rep =  db.videogames.aggregate([
    { "$match": { "Genre": "Racing" } },
    { "$sort": { "Global_Sales": -1 }},
    {
        "$group": {
            "_id": "$Year_of_Release",
            "TotalSales": {"$sum": "$Global_Sales"},
            "Games": { 
                "$push": {
                    "Name": "$Name",
                    "Publisher": "$Publisher",
                    "Sales": "$Global_Sales",
                }
            },
        }
    },
    { 
        "$project": {
            "_id": 0,
            "Year": "$_id",
            "TotalSales": 1,
            "Games": 1
            }
    },
    { "$sort": { "TotalSales": -1 }},
    {
        "$limit": 10
    }

])
for r in rep:
    pp(r)
"""
rep =  db.videogames.aggregate([
    { "$match": { "Genre": "Racing" } },
    {
        "$setWindowFields": {
            "partitionBy": "$Year_of_Release",
            "sortBy": { "Year_of_Release": -1 },
            "output": {
                "cumulativeSales": {
                    "$sum": "$Global_Sales",
                    "window": {
                        "documents": ["unbounded", "current"]
                    }
                }
            } 
        }
    },
    { 
        "$project": {
            "_id": 0,
            "Year_of_Release": 1,
            "Name": 1,
            "cumulativeSales": 1
            }
    },{
        "$group": {
            "_id": "$Year_of_Release",
            "TotalSales": {"$max": "$Global_Sales"},
        }
    },
    {
        "$limit": 10
    }

])
for r in rep:
    pp(r)

"""


{'Games': [{'Name': 'Mario Kart Wii', 'Publisher': 'Nintendo', 'Sales': 35.52},
           {'Name': 'Pure',
            'Publisher': 'Disney Interactive Studios',
            'Sales': 2.47},
           {'Name': 'Midnight Club: Los Angeles',
            'Publisher': 'Take-Two Interactive',
            'Sales': 2.45},
           {'Name': 'Midnight Club: Los Angeles',
            'Publisher': 'Take-Two Interactive',
            'Sales': 2.35},
           {'Name': 'Need For Speed: Undercover',
            'Publisher': 'Electronic Arts',
            'Sales': 2.05},
           {'Name': 'Burnout Paradise',
            'Publisher': 'Electronic Arts',
            'Sales': 1.94},
           {'Name': 'Need For Speed: Undercover',
            'Publisher': 'Electronic Arts',
            'Sales': 1.68},
           {'Name': 'Need For Speed: Undercover',
            'Publisher': 'Electronic Arts',
            'Sales': 1.4},
           {'Name': 'Burnout Paradise',
            'Publisher': 'Electronic A

'\nrep =  db.videogames.aggregate([\n    { "$match": { "Genre": "Racing" } },\n    {\n        "$setWindowFields": {\n            "partitionBy": "$Year_of_Release",\n            "sortBy": { "Year_of_Release": -1 },\n            "output": {\n                "cumulativeSales": {\n                    "$sum": "$Global_Sales",\n                    "window": {\n                        "documents": ["unbounded", "current"]\n                    }\n                }\n            } \n        }\n    },\n    { \n        "$project": {\n            "_id": 0,\n            "Year_of_Release": 1,\n            "Name": 1,\n            "cumulativeSales": 1\n            }\n    },{\n        "$group": {\n            "_id": "$Year_of_Release",\n            "TotalSales": {"$max": "$Global_Sales"},\n        }\n    },\n    {\n        "$limit": 10\n    }\n\n])\nfor r in rep:\n    pp(r)\n\n'

# Les publishers avec le plus de ventes de jeux de racing

In [77]:
rep =  db.videogames.aggregate([
    { "$match": { "Genre": "Racing" } },
    {
        "$group": {
            "_id": "$Publisher",
            "TotalSales": {"$sum": "$Global_Sales"},
            "TotalNA": {"$sum": "$NA_Sales"},
            "TotalEU": {"$sum": "$EU_Sales"},
            "TotalJP": {"$sum": "$JP_Sales"},
        }
    },
    { "$project": {
        "_id": 0,
        "Publisher": "$_id",
        "TotalSales": 1,
        "TotalNA": 1,
        "TotalEU": 1,
        "TotalJP": 1,
    }},
    {'$sort': {"TotalSales": -1}},
    {
        "$limit": 5
    }
])
for r in rep:
    print(r)

{'TotalSales': 151.3, 'TotalNA': 73.54, 'TotalEU': 39.71, 'TotalJP': 29.25, 'Publisher': 'Nintendo'}
{'TotalSales': 144.77, 'TotalNA': 75.42, 'TotalEU': 50.64, 'TotalJP': 1.2, 'Publisher': 'Electronic Arts'}
{'TotalSales': 109.96, 'TotalNA': 42.449999999999996, 'TotalEU': 35.15, 'TotalJP': 13.89, 'Publisher': 'Sony Computer Entertainment'}
{'TotalSales': 39.97, 'TotalNA': 27.32, 'TotalEU': 8.96, 'TotalJP': 0.02, 'Publisher': 'THQ'}
{'TotalSales': 34.14, 'TotalNA': 6.24, 'TotalEU': 22.53, 'TotalJP': 0.65, 'Publisher': 'Codemasters'}


# La région avec le plus de ventes de jeux de racing

In [73]:
rep =  db.videogames.aggregate([
    { "$match": { "Genre": "Racing" } },
    {
        "$group": {
            "_id": "null",
            "TotalNA": {"$sum": "$NA_Sales"},
            "TotalEU": {"$sum": "$EU_Sales"},
            "TotalJP": {"$sum": "$JP_Sales"},
        }
    },
    { "$project": {
        "TotalNA": 1,
        "TotalEU": 1,
        "TotalJP": 1,
    }},
])
for r in rep:
    print(r)

{'_id': 'null', 'TotalNA': 359.35, 'TotalEU': 236.51, 'TotalJP': 56.71}


# La région avec le plus de ventes de jeux de racing par année, pour les 5 dernières années

In [81]:
rep =  db.videogames.aggregate([
    { "$match": { "Genre": "Racing" } },
    {
        "$group": {
            "_id": "$Year_of_Release",
            "TotalSales": {"$sum": "$Global_Sales"},
            "TotalNA": {"$sum": "$NA_Sales"},
            "TotalEU": {"$sum": "$EU_Sales"},
            "TotalJP": {"$sum": "$JP_Sales"},
        }
    },
    { "$project": {
        "_id": 0,
        "Year": "$_id",
        "TotalSales": 1,
        "TotalNA": 1,
        "TotalEU": 1,
        "TotalJP": 1,
    }},
    {
        "$sort": { "Year": -1}
    },
    {
        "$limit": 10
    }
])
for r in rep:
    print(r)

{'TotalSales': 2.83, 'TotalNA': 0.73, 'TotalEU': 1.77, 'TotalJP': 0.01, 'Year': 2016.0}
{'TotalSales': 8.07, 'TotalNA': 2.09, 'TotalEU': 4.75, 'TotalJP': 0.28, 'Year': 2015.0}
{'TotalSales': 16.64, 'TotalNA': 5.92, 'TotalEU': 7.61, 'TotalJP': 1.47, 'Year': 2014.0}
{'TotalSales': 12.33, 'TotalNA': 4.22, 'TotalEU': 6.06, 'TotalJP': 0.54, 'Year': 2013.0}
{'TotalSales': 13.68, 'TotalNA': 4.26, 'TotalEU': 7.1000000000000005, 'TotalJP': 0.2, 'Year': 2012.0}
{'TotalSales': 34.83, 'TotalNA': 14.76, 'TotalEU': 13.29, 'TotalJP': 3.14, 'Year': 2011.0}
{'TotalSales': 34.36, 'TotalNA': 14.13, 'TotalEU': 14.17, 'TotalJP': 1.04, 'Year': 2010.0}
{'TotalSales': 33.65, 'TotalNA': 14.93, 'TotalEU': 12.879999999999999, 'TotalJP': 0.5, 'Year': 2009.0}
{'TotalSales': 70.03, 'TotalNA': 33.25, 'TotalEU': 23.99, 'TotalJP': 4.21, 'Year': 2008.0}
{'TotalSales': 38.87, 'TotalNA': 18.39, 'TotalEU': 13.01, 'TotalJP': 0.94, 'Year': 2007.0}
